In [1]:
#importing dependencies

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import save_image

#import _pickle as cpickle #to store model histories in a file
import os
import imageio
from PIL import Image

use_cuda = False
device   = torch.device('cpu')
if torch.cuda.is_available():
    use_cuda = True
    device   = torch.device('cuda')
print(use_cuda)    
    
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots"

True


In [2]:
#specifying parameters
image_size = 64 #HACK to use MNIST architecture
G_input_dim = 100
G_output_dim = 3
D_input_dim = 3
D_output_dim = 1
num_filters = [1024, 512, 256, 128]

learning_rate = 0.0002
betas = (0.5, 0.999)
batch_size = 128
num_epochs = 1000

data_dir = './Train_data'
save_dir = './DCGAN_results/'

In [3]:
print(os.getcwd())

/home/shreyashpandey/PoseGuided


In [4]:
#loading data

transform = transforms.Compose([
                                 #Hack to make MNIST code work
                                transforms.ToTensor(),
                                #transforms.Normalize(mean=(214.0466981, 206.55220904, 203.99178198), std=(54.34939265, 55.62690195, 58.85794001))
                               ])
                                

df_data = dsets.ImageFolder(data_dir, transform = transform)

data_loader = torch.utils.data.DataLoader(dataset=df_data,
                                          batch_size=batch_size,
                                          shuffle=True)


## VAE Model

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(64*64*3, 4096)
        self.fc2 = nn.Linear(4096,1000)
        self.fc3 = nn.Linear(1000,400)
        self.fc31 = nn.Linear(400, 60)
        self.fc32 = nn.Linear(400, 60)
        self.fc4 = nn.Linear(60, 400)
        self.fc5 = nn.Linear(400,1000)
        self.fc6 = nn.Linear(1000,4096)
        self.fc7 = nn.Linear(4096, 64*64*3)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        return self.fc31(h3), self.fc32(h3)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h4 = F.relu(self.fc4(z))
        h5 = F.relu(self.fc5(h4))
        h6 = F.relu(self.fc6(h5))
        #out = F.sigmoid(self.fc7(h6))
        out = self.fc7(h6)
        return out

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 64*64*3))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

##  Loss Function

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x.view(-1, 64*64*3), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

## Set up training and testing functions

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(data_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model.forward(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(data_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(data_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0 and epoch%1 == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(-1, 3, 64, 64)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(data_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


# Train and Test

In [8]:
for epoch in range(1, num_epochs+1):
    train(epoch)
    test(epoch)
    
    with torch.no_grad():
        if epoch%1 == 0:
            sample = torch.randn(64, 60).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 3, 64, 64),'results/sample_' + str(epoch) + '.png')


Train Epoch: 1 [0/32031 (0%)]	Loss: 8949.535156
Train Epoch: 1 [1280/32031 (4%)]	Loss: 8865.382812
Train Epoch: 1 [2560/32031 (8%)]	Loss: 21178.947266
Train Epoch: 1 [3840/32031 (12%)]	Loss: 6802.149902
Train Epoch: 1 [5120/32031 (16%)]	Loss: 1289.246094
Train Epoch: 1 [6400/32031 (20%)]	Loss: 917.844910
Train Epoch: 1 [7680/32031 (24%)]	Loss: 670.380005
Train Epoch: 1 [8960/32031 (28%)]	Loss: 602.376465
Train Epoch: 1 [10240/32031 (32%)]	Loss: 562.315796
Train Epoch: 1 [11520/32031 (36%)]	Loss: 509.550293
Train Epoch: 1 [12800/32031 (40%)]	Loss: 521.720459
Train Epoch: 1 [14080/32031 (44%)]	Loss: 537.670349
Train Epoch: 1 [15360/32031 (48%)]	Loss: 518.485291
Train Epoch: 1 [16640/32031 (52%)]	Loss: 484.445862
Train Epoch: 1 [17920/32031 (56%)]	Loss: 513.425903
Train Epoch: 1 [19200/32031 (60%)]	Loss: 504.212189
Train Epoch: 1 [20480/32031 (64%)]	Loss: 481.350250
Train Epoch: 1 [21760/32031 (68%)]	Loss: 493.680145
Train Epoch: 1 [23040/32031 (72%)]	Loss: 501.151184
Train Epoch: 1 [2432

Train Epoch: 6 [29440/32031 (92%)]	Loss: 325.621277
Train Epoch: 6 [30720/32031 (96%)]	Loss: 320.515991
Train Epoch: 6 [7750/32031 (100%)]	Loss: 330.263168
====> Epoch: 6 Average loss: 330.9819
====> Test set loss: 344.5012
Train Epoch: 7 [0/32031 (0%)]	Loss: 355.120483
Train Epoch: 7 [1280/32031 (4%)]	Loss: 333.705780
Train Epoch: 7 [2560/32031 (8%)]	Loss: 313.381439
Train Epoch: 7 [3840/32031 (12%)]	Loss: 305.490601
Train Epoch: 7 [5120/32031 (16%)]	Loss: 318.341919
Train Epoch: 7 [6400/32031 (20%)]	Loss: 327.258453
Train Epoch: 7 [7680/32031 (24%)]	Loss: 319.084869
Train Epoch: 7 [8960/32031 (28%)]	Loss: 316.919434
Train Epoch: 7 [10240/32031 (32%)]	Loss: 320.431824
Train Epoch: 7 [11520/32031 (36%)]	Loss: 332.674255
Train Epoch: 7 [12800/32031 (40%)]	Loss: 324.125946
Train Epoch: 7 [14080/32031 (44%)]	Loss: 323.433685
Train Epoch: 7 [15360/32031 (48%)]	Loss: 319.446747
Train Epoch: 7 [16640/32031 (52%)]	Loss: 328.718292
Train Epoch: 7 [17920/32031 (56%)]	Loss: 328.876404
Train Epoc

Train Epoch: 12 [21760/32031 (68%)]	Loss: 289.724670
Train Epoch: 12 [23040/32031 (72%)]	Loss: 304.851807
Train Epoch: 12 [24320/32031 (76%)]	Loss: 307.777832
Train Epoch: 12 [25600/32031 (80%)]	Loss: 302.965454
Train Epoch: 12 [26880/32031 (84%)]	Loss: 292.791992
Train Epoch: 12 [28160/32031 (88%)]	Loss: 294.424500
Train Epoch: 12 [29440/32031 (92%)]	Loss: 309.606262
Train Epoch: 12 [30720/32031 (96%)]	Loss: 310.766937
Train Epoch: 12 [7750/32031 (100%)]	Loss: 260.013262
====> Epoch: 12 Average loss: 300.2108
====> Test set loss: 297.3886
Train Epoch: 13 [0/32031 (0%)]	Loss: 284.801544
Train Epoch: 13 [1280/32031 (4%)]	Loss: 297.667847
Train Epoch: 13 [2560/32031 (8%)]	Loss: 293.782440
Train Epoch: 13 [3840/32031 (12%)]	Loss: 281.318085
Train Epoch: 13 [5120/32031 (16%)]	Loss: 281.117920
Train Epoch: 13 [6400/32031 (20%)]	Loss: 307.311035
Train Epoch: 13 [7680/32031 (24%)]	Loss: 286.754700
Train Epoch: 13 [8960/32031 (28%)]	Loss: 305.650360
Train Epoch: 13 [10240/32031 (32%)]	Loss: 30

Train Epoch: 19 [1280/32031 (4%)]	Loss: 275.049072
Train Epoch: 19 [2560/32031 (8%)]	Loss: 255.114014
Train Epoch: 19 [3840/32031 (12%)]	Loss: 269.161255
Train Epoch: 19 [5120/32031 (16%)]	Loss: 257.389496
Train Epoch: 19 [6400/32031 (20%)]	Loss: 293.263275
Train Epoch: 19 [7680/32031 (24%)]	Loss: 282.462860
Train Epoch: 19 [8960/32031 (28%)]	Loss: 281.212738
Train Epoch: 19 [10240/32031 (32%)]	Loss: 290.877075
Train Epoch: 19 [11520/32031 (36%)]	Loss: 274.673248
Train Epoch: 19 [12800/32031 (40%)]	Loss: 289.820892
Train Epoch: 19 [14080/32031 (44%)]	Loss: 274.904266
Train Epoch: 19 [15360/32031 (48%)]	Loss: 274.722351
Train Epoch: 19 [16640/32031 (52%)]	Loss: 274.510406
Train Epoch: 19 [17920/32031 (56%)]	Loss: 295.228668
Train Epoch: 19 [19200/32031 (60%)]	Loss: 285.307587
Train Epoch: 19 [20480/32031 (64%)]	Loss: 257.402985
Train Epoch: 19 [21760/32031 (68%)]	Loss: 287.267517
Train Epoch: 19 [23040/32031 (72%)]	Loss: 275.571716
Train Epoch: 19 [24320/32031 (76%)]	Loss: 262.934082
Tr

Train Epoch: 24 [26880/32031 (84%)]	Loss: 265.471863
Train Epoch: 24 [28160/32031 (88%)]	Loss: 272.371674
Train Epoch: 24 [29440/32031 (92%)]	Loss: 273.714661
Train Epoch: 24 [30720/32031 (96%)]	Loss: 284.325226
Train Epoch: 24 [7750/32031 (100%)]	Loss: 247.724294
====> Epoch: 24 Average loss: 269.1417
====> Test set loss: 280.7639
Train Epoch: 25 [0/32031 (0%)]	Loss: 276.443115
Train Epoch: 25 [1280/32031 (4%)]	Loss: 264.593658
Train Epoch: 25 [2560/32031 (8%)]	Loss: 264.617981
Train Epoch: 25 [3840/32031 (12%)]	Loss: 267.547821
Train Epoch: 25 [5120/32031 (16%)]	Loss: 276.561859
Train Epoch: 25 [6400/32031 (20%)]	Loss: 280.339294
Train Epoch: 25 [7680/32031 (24%)]	Loss: 267.592346
Train Epoch: 25 [8960/32031 (28%)]	Loss: 273.384613
Train Epoch: 25 [10240/32031 (32%)]	Loss: 267.262390
Train Epoch: 25 [11520/32031 (36%)]	Loss: 257.792419
Train Epoch: 25 [12800/32031 (40%)]	Loss: 267.885193
Train Epoch: 25 [14080/32031 (44%)]	Loss: 261.064148
Train Epoch: 25 [15360/32031 (48%)]	Loss: 27

...

Train Epoch: 417 [19200/32031 (60%)]	Loss: 207.053635
Train Epoch: 417 [20480/32031 (64%)]	Loss: 195.341232
Train Epoch: 417 [21760/32031 (68%)]	Loss: 200.186127
Train Epoch: 417 [23040/32031 (72%)]	Loss: 200.091629
Train Epoch: 417 [24320/32031 (76%)]	Loss: 200.838333
Train Epoch: 417 [25600/32031 (80%)]	Loss: 191.484009
Train Epoch: 417 [26880/32031 (84%)]	Loss: 204.110794
Train Epoch: 417 [28160/32031 (88%)]	Loss: 201.935791
Train Epoch: 417 [29440/32031 (92%)]	Loss: 206.825760
Train Epoch: 417 [30720/32031 (96%)]	Loss: 213.134583
Train Epoch: 417 [7750/32031 (100%)]	Loss: 194.095388
====> Epoch: 417 Average loss: 200.6700
====> Test set loss: 195.1790
Train Epoch: 418 [0/32031 (0%)]	Loss: 201.209732
Train Epoch: 418 [1280/32031 (4%)]	Loss: 197.052200
Train Epoch: 418 [2560/32031 (8%)]	Loss: 189.314819
Train Epoch: 418 [3840/32031 (12%)]	Loss: 194.710419
Train Epoch: 418 [5120/32031 (16%)]	Loss: 206.868134
Train Epoch: 418 [6400/32031 (20%)]	Loss: 202.506516
Train Epoch: 418 [7680/3

Train Epoch: 423 [6400/32031 (20%)]	Loss: 197.642914
Train Epoch: 423 [7680/32031 (24%)]	Loss: 191.740219
Train Epoch: 423 [8960/32031 (28%)]	Loss: 189.932663
Train Epoch: 423 [10240/32031 (32%)]	Loss: 199.767090
Train Epoch: 423 [11520/32031 (36%)]	Loss: 197.365479
Train Epoch: 423 [12800/32031 (40%)]	Loss: 195.910919
Train Epoch: 423 [14080/32031 (44%)]	Loss: 208.192932
Train Epoch: 423 [15360/32031 (48%)]	Loss: 206.866837
Train Epoch: 423 [16640/32031 (52%)]	Loss: 198.375168
Train Epoch: 423 [17920/32031 (56%)]	Loss: 193.457581
Train Epoch: 423 [19200/32031 (60%)]	Loss: 199.531372
Train Epoch: 423 [20480/32031 (64%)]	Loss: 194.508621
Train Epoch: 423 [21760/32031 (68%)]	Loss: 198.022629
Train Epoch: 423 [23040/32031 (72%)]	Loss: 212.640045
Train Epoch: 423 [24320/32031 (76%)]	Loss: 206.384430
Train Epoch: 423 [25600/32031 (80%)]	Loss: 218.287170
Train Epoch: 423 [26880/32031 (84%)]	Loss: 199.696487
Train Epoch: 423 [28160/32031 (88%)]	Loss: 199.046951
Train Epoch: 423 [29440/32031 (

Train Epoch: 428 [28160/32031 (88%)]	Loss: 204.920288
Train Epoch: 428 [29440/32031 (92%)]	Loss: 203.620422
Train Epoch: 428 [30720/32031 (96%)]	Loss: 202.004898
Train Epoch: 428 [7750/32031 (100%)]	Loss: 186.258474
====> Epoch: 428 Average loss: 201.5154
====> Test set loss: 194.0026
Train Epoch: 429 [0/32031 (0%)]	Loss: 195.012939
Train Epoch: 429 [1280/32031 (4%)]	Loss: 194.498932
Train Epoch: 429 [2560/32031 (8%)]	Loss: 197.083420
Train Epoch: 429 [3840/32031 (12%)]	Loss: 202.567108
Train Epoch: 429 [5120/32031 (16%)]	Loss: 205.592117
Train Epoch: 429 [6400/32031 (20%)]	Loss: 197.008041
Train Epoch: 429 [7680/32031 (24%)]	Loss: 196.485291
Train Epoch: 429 [8960/32031 (28%)]	Loss: 200.464783
Train Epoch: 429 [10240/32031 (32%)]	Loss: 205.812469
Train Epoch: 429 [11520/32031 (36%)]	Loss: 201.505325
Train Epoch: 429 [12800/32031 (40%)]	Loss: 206.225128
Train Epoch: 429 [14080/32031 (44%)]	Loss: 201.497498
Train Epoch: 429 [15360/32031 (48%)]	Loss: 206.014114
Train Epoch: 429 [16640/32

Train Epoch: 434 [15360/32031 (48%)]	Loss: 193.389862
Train Epoch: 434 [16640/32031 (52%)]	Loss: 202.034058
Train Epoch: 434 [17920/32031 (56%)]	Loss: 190.683380
Train Epoch: 434 [19200/32031 (60%)]	Loss: 214.145752
Train Epoch: 434 [20480/32031 (64%)]	Loss: 211.395630
Train Epoch: 434 [21760/32031 (68%)]	Loss: 199.401123
Train Epoch: 434 [23040/32031 (72%)]	Loss: 206.153625
Train Epoch: 434 [24320/32031 (76%)]	Loss: 206.074173
Train Epoch: 434 [25600/32031 (80%)]	Loss: 202.254181
Train Epoch: 434 [26880/32031 (84%)]	Loss: 205.171356
Train Epoch: 434 [28160/32031 (88%)]	Loss: 188.426346
Train Epoch: 434 [29440/32031 (92%)]	Loss: 208.817444
Train Epoch: 434 [30720/32031 (96%)]	Loss: 216.371246
Train Epoch: 434 [7750/32031 (100%)]	Loss: 193.028415
====> Epoch: 434 Average loss: 201.0636


KeyboardInterrupt: 